In [1]:
!git clone https://dagshub.com/Omdena/AntananarivoMadagascarChapter_FakeJobPostings.git

Cloning into 'AntananarivoMadagascarChapter_FakeJobPostings'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 174 (delta 78), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (174/174), 15.19 MiB | 4.90 MiB/s, done.
Resolving deltas: 100% (78/78), done.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string

#import contractions # I'm you'r etc
import nltk
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, classification_report,
                             confusion_matrix,roc_curve, roc_auc_score, ConfusionMatrixDisplay)

from imblearn.over_sampling import RandomOverSampler

from tqdm import tqdm # printing the status bar
from bs4 import BeautifulSoup

import requests
from io import BytesIO
from zipfile import ZipFile

import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)


from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
# Define the URL of the zip file
zip_url = "https://github.com/alberto2020china/fake_jobs/raw/main/data/fake_job_postings.csv.zip"

# Send a GET request to the URL
response = requests.get(zip_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content of the zip file
    with ZipFile(BytesIO(response.content)) as zip_file:
        # Specify the file you want to extract from the zip archive
        file_to_extract = "fake_job_postings.csv"

        # Extract the file and load it into a Pandas DataFrame (assuming it's a CSV file)
        with zip_file.open(file_to_extract) as file:
            # Now you can use the file (e.g., load it into a Pandas DataFrame)
            # For example:
            # import pandas as pd
            df_raw = pd.read_csv(file)
            print(f"{file_to_extract} has been successfully loaded.")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

fake_job_postings.csv has been successfully loaded.


In [ ]:
X = df_raw.drop('fraudulent', axis=1)
y = df_raw['fraudulent']

# Define the undersampler
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)

# Perform undersampling
X_resampled, y_resampled = undersampler.fit_resample(X, y)

# Combine the undersampled data back into a DataFrame
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='fraudulent')], axis=1)
data = df_resampled.copy()

df_train = data[["title","description","fraudulent"]]
df_train['textual_input']=data['title']+data['description']

In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039278 sha256=6c2d5d2b458bfec8abdcddf6dc2b2bd21f9419f862066558daf5bf739af59604
  Stored in dir

In [ ]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2")

In [ ]:
df_train.dropna(subset=["description"], inplace=True)

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [ ]:
from tqdm import tqdm
import re
import string
from bs4 import BeautifulSoup
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def preprocess(corpus):
    preprocessed = []
    for sentance in tqdm(corpus):
        #sentance = re.sub(r"http+", "", sentance)
        sentance = re.sub(r"http\S+", "", sentance)
        sentance = BeautifulSoup(sentance, 'lxml').get_text()
        sentance = decontracted(sentance)
        sentance = re.sub("\S*\d\S*", "", sentance).strip()
        sentance = re.sub('[^A-Za-z]+', ' ', sentance)
        sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
        preprocessed.append(sentance.strip())

    return preprocessed

In [ ]:
df_train["textual_input_transformed"] = preprocess(df_train['textual_input'].values)

100%|██████████| 1731/1731 [00:02<00:00, 682.20it/s]


In [ ]:
topics, probs = topic_model.fit_transform(df_train["textual_input_transformed"])

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,502,-1_team_work_new_business,"[team, work, new, business, sales, product, co...",[customer success district managerthis company...
1,0,77,0_administrative_assistant_office_receptionist,"[administrative, assistant, office, receptioni...",[admin assistant txjob descriptionadministrati...
2,1,73,1_software_java_support_technical,"[software, java, support, technical, engineer,...",[software engineer java edi developersoftware ...
3,2,69,2_solutions_project_gas_oil,"[solutions, project, gas, oil, engineering, de...",[project subcontracting managercorporate overv...
4,3,62,3_customer_service_call_center,"[customer, service, call, center, customers, c...",[customer service representativewe seeking can...
5,4,62,4_customer_mail_service_delivery,"[customer, mail, service, delivery, site, inco...",[customer service associate customer service a...
6,5,44,5_front_end_web_javascript,"[front, end, web, javascript, developer, php, ...",[senior front end developermindworks digital a...
7,6,40,6_cash_time_start_extra,"[cash, time, start, extra, part, day, free, ea...",[weekly part time cash job staff required week...
8,7,32,7_project_process_client_schedule,"[project, process, client, schedule, develop, ...",[project manager mechanicalessential duties re...
9,8,31,8_wages_figure_entry_legitimate,"[wages, figure, entry, legitimate, typing, ran...",[data entry clerkwe seeking individuals connec...


In [ ]:
topic_model.get_topic(topic=0)

[('administrative', 0.06859523337523749),
 ('assistant', 0.05443697355967925),
 ('office', 0.04047383016586156),
 ('receptionist', 0.02741864192571623),
 ('duties', 0.02730612661711839),
 ('skills', 0.022526873850808073),
 ('answering', 0.022362288932367063),
 ('filing', 0.017367043931560622),
 ('church', 0.01707972332786944),
 ('multi', 0.017074787265373953)]

In [ ]:
topic_model.get_topic(topic=1)

[('software', 0.027110202674842983),
 ('java', 0.023246881400638606),
 ('support', 0.0208883306332963),
 ('technical', 0.01988132441619987),
 ('engineer', 0.01962397161716408),
 ('data', 0.017342001670691727),
 ('development', 0.01730870383871033),
 ('database', 0.01704011238713689),
 ('experience', 0.015304631793835927),
 ('systems', 0.015151418326097711)]

In [ ]:
topic_model.get_representative_docs(0)

['admin assistant txjob descriptionadministrative assistantdescriptionan insurance consulting practice currently seeking skilled administrative assistant join team administrative assistant provide administrative clerical support insurance compliance department success role depend upon ability manage multiple tasks meet deadlines provide outstanding administrative customer service support job responsibilities include heavy data entryscanning imaging documentsother duties assigned department managerqualifications business communication skills written verbal advanced level pc skills using microsoft office word excel access powerpoint outlook superior customer service skills must willing work holidays weekends needed individual must self motivated excellent time management attendance organizational multi tasking abilities',
 'admin assistantjob description administrative assistantdescriptionan insurance consulting practice currently seeking skilled administrative assistant join team admini

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_term_rank()

In [ ]:
#topic_model.set_topic_labels({0:"ffff",1:"hhhh"})
#topic_model.reduce_topics(abs,nr_topics=100)
#topic_model.merge_topics(abs,topics_to_merge=[100,12])

In [ ]:
%cd /content/ss/AntananarivoMadagascarChapter_FakeJobPostings

/content/ss/AntananarivoMadagascarChapter_FakeJobPostings


In [ ]:
!git config --global user.email "chaimatrabelsi889@gmail.com"
!git config  --global user.name "chaimatrabelsi889"

In [ ]:
!git add .
!git commit -m "Visualization_using_BERTopic"
!git push

[main 1672893] Visualization_using_BERTopic
 1 file changed, 1 insertion(+)
 create mode 100644 code/Visualisation_TopicModeling.ipynb
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 59.39 KiB | 4.24 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
fatal: could not read Username for 'https://dagshub.com': No such device or address
send-pack: unexpected disconnect while reading sideband packet
fatal: the remote end hung up unexpectedly
error: failed to push some refs to 'https://dagshub.com/Omdena/AntananarivoMadagascarChapter_FakeJobPostings.git'
